# 🚀 Multi-Agent Coding System - Comprehensive Demo

Welcome to the complete demonstration of the Multi-Agent Coding System! This notebook will walk you through all the ways to use this powerful AI-driven development platform.

## 📋 What You'll Learn

1. **System Overview** - Understanding the architecture
2. **Basic Usage** - Simple examples to get started
3. **API Integration** - RESTful API usage
4. **Advanced Workflows** - TDD, Full, and Individual steps
5. **Real-World Examples** - Practical applications
6. **Docker Deployment** - Containerized usage
7. **Testing & Monitoring** - Quality assurance and tracking

---

## 🏗️ System Architecture

The Multi-Agent Coding System consists of 6 specialized AI agents working together:

| Agent | Role | Responsibilities |
|-------|------|------------------|
| **🎯 Planner** | Project Architect | Breaks down requirements into actionable tasks |
| **🎨 Designer** | System Designer | Creates technical designs and architecture |
| **💻 Coder** | Implementation Expert | Writes the actual code |
| **🧪 Test Writer** | Quality Assurance | Creates comprehensive test suites |
| **👁️ Reviewer** | Code Reviewer | Reviews and approves/rejects agent outputs |
| **🚀 Executor** | Runtime Manager | Executes code in Docker containers |

These agents collaborate through different workflows to transform your requirements into working code.

## 🔧 Setup & Prerequisites

Let's start by checking our environment and setting up everything we need.

In [2]:
%pip install -r requirements.txt
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 5.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [ipywidgets]3 [ipywidgets]
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Import required libraries
import os
import sys
import json
import time
import asyncio
import subprocess
from datetime import datetime
from pathlib import Path

# For API calls
import requests
import httpx

# For rich output
from IPython.display import display, HTML, Markdown, Code, JSON
from IPython.display import clear_output
import ipywidgets as widgets

# For progress tracking
from tqdm.notebook import tqdm

# Check Python version
print(f"Python version: {sys.version}")
print(f"Current directory: {os.getcwd()}")

ModuleNotFoundError: No module named 'ipywidgets'

In [ ]:
# Helper functions for beautiful output
def success(message):
    display(HTML(f'<div style="color: green; font-weight: bold;">✅ {message}</div>'))

def error(message):
    display(HTML(f'<div style="color: red; font-weight: bold;">❌ {message}</div>'))

def info(message):
    display(HTML(f'<div style="color: blue; font-weight: bold;">ℹ️ {message}</div>'))

def warning(message):
    display(HTML(f'<div style="color: orange; font-weight: bold;">⚠️ {message}</div>'))

def code_block(code, language="python"):
    display(Code(code, language=language))

In [ ]:
# Check if required services are running
def check_service(url, name):
    try:
        response = requests.get(url, timeout=2)
        if response.status_code == 200:
            success(f"{name} is running at {url}")
            return True
    except:
        pass
    error(f"{name} is not running. Please start it first.")
    return False

# Check services
orchestrator_running = check_service("http://localhost:8080/health", "Orchestrator")
api_running = check_service("http://localhost:8000/health", "API")

if not orchestrator_running:
    info("Start the orchestrator with: python orchestrator/orchestrator_agent.py")
if not api_running:
    info("Start the API with: python api/orchestrator_api.py")

## 🎯 Basic Usage - Python Scripts

Let's start with the simplest way to use the system - through Python scripts.

### 1. Hello World Example

The simplest possible example - creating a "Hello World" function with tests.

In [ ]:
# Simulate the hello_agents.py script
hello_world_code = '''
import subprocess
import sys

# Define the task
task = "Create a Python function that returns 'Hello, World!' with a test"

# Run the simple example with TDD workflow
result = subprocess.run(
    [sys.executable, "simple_example.py", "--workflow", "tdd", "--task", task],
    capture_output=True,
    text=True
)

print(result.stdout)
'''

info("This would execute the hello world example:")
code_block(hello_world_code)

### 2. Different Workflow Types

The system supports multiple workflow types, each optimized for different scenarios:

In [ ]:
# Display workflow comparison
workflow_data = {
    "Workflow": ["TDD", "Full", "Planning", "Design", "Implementation"],
    "Description": [
        "Test-Driven Development - Tests first, then code",
        "Complete workflow - All phases without test-first",
        "Planning only - Break down requirements",
        "Design only - Create technical architecture",
        "Implementation only - Direct coding"
    ],
    "Best For": [
        "Quality-critical code, APIs, libraries",
        "General application development",
        "Complex projects needing strategy",
        "System architecture, database design",
        "Simple scripts, prototypes"
    ],
    "Agents Used": [
        "All 6 agents",
        "5 agents (no test writer first)",
        "Planner + Reviewer",
        "Designer + Reviewer",
        "Coder + Reviewer"
    ]
}

import pandas as pd
df = pd.DataFrame(workflow_data)
display(HTML(df.to_html(index=False, classes="table table-striped")))

## 🌐 REST API Usage

The system provides a comprehensive REST API for integration with other applications.

In [ ]:
# API Base URL
API_BASE = "http://localhost:8000"

# Check available workflow types
if api_running:
    response = requests.get(f"{API_BASE}/workflow-types")
    if response.status_code == 200:
        info("Available workflow types:")
        display(JSON(response.json()))
else:
    warning("API is not running. Start it with: python api/orchestrator_api.py")

### Submitting a Workflow Request

In [ ]:
# Example: Submit a simple coding request
def submit_workflow(requirements, workflow_type="full"):
    """Submit a workflow request to the API"""
    
    if not api_running:
        error("API is not running!")
        return None
    
    payload = {
        "requirements": requirements,
        "workflow_type": workflow_type,
        "max_retries": 3,
        "timeout_seconds": 300
    }
    
    response = requests.post(f"{API_BASE}/execute-workflow", json=payload)
    
    if response.status_code == 200:
        result = response.json()
        success(f"Workflow submitted! Session ID: {result['session_id']}")
        return result['session_id']
    else:
        error(f"Failed to submit workflow: {response.status_code}")
        return None

# Example submission
info("Example API call:")
code_block('''
session_id = submit_workflow(
    requirements="Create a Python function to calculate fibonacci numbers",
    workflow_type="tdd"
)
''')

### Monitoring Workflow Progress

In [ ]:
async def monitor_workflow_progress(session_id, max_wait=300):
    """Monitor workflow progress with live updates"""
    
    start_time = time.time()
    status_widget = widgets.Output()
    progress_bar = tqdm(total=100, desc="Workflow Progress")
    
    display(status_widget)
    
    async with httpx.AsyncClient() as client:
        while True:
            elapsed = time.time() - start_time
            if elapsed > max_wait:
                error("Workflow timed out!")
                break
            
            try:
                response = await client.get(f"{API_BASE}/workflow-status/{session_id}")
                if response.status_code == 200:
                    data = response.json()
                    
                    with status_widget:
                        clear_output(wait=True)
                        info(f"Status: {data['status']}")
                        
                        if data.get('progress'):
                            current = data['progress'].get('current_step', 0)
                            total = data['progress'].get('total_steps', 1)
                            progress_bar.n = int((current / total) * 100)
                            progress_bar.refresh()
                    
                    if data['status'] in ['completed', 'failed']:
                        progress_bar.n = 100 if data['status'] == 'completed' else progress_bar.n
                        progress_bar.close()
                        
                        if data['status'] == 'completed':
                            success("Workflow completed successfully!")
                            return data
                        else:
                            error(f"Workflow failed: {data.get('error', 'Unknown error')}")
                            return None
                            
            except Exception as e:
                error(f"Error checking status: {e}")
            
            await asyncio.sleep(2)

# Example usage
info("Example workflow monitoring:")
code_block('''
# Submit and monitor a workflow
session_id = submit_workflow("Create a calculator class", "tdd")
if session_id:
    result = await monitor_workflow_progress(session_id)
    if result:
        print(json.dumps(result, indent=2))
''')

## 🔄 Real-World Examples

Let's explore some practical examples of what you can build with the Multi-Agent Coding System.

### Example 1: Building a REST API

In [ ]:
# REST API Example
rest_api_requirements = """
Create a REST API for a book library management system with the following features:
1. CRUD operations for books (title, author, ISBN, year)
2. Search functionality by title or author
3. Pagination support
4. Input validation
5. Error handling
6. Use FastAPI framework
7. Include comprehensive tests
"""

info("Example: Building a REST API")
code_block(f'requirements = """{rest_api_requirements}"""\n\n# Use TDD workflow for API development\nsession_id = submit_workflow(requirements, "tdd")')

### Example 2: Creating a CLI Tool

In [ ]:
# CLI Tool Example
cli_tool_requirements = """
Build a command-line file organizer tool that:
1. Scans a directory for files
2. Organizes files by type (documents, images, videos, etc.)
3. Supports custom organization rules
4. Has dry-run mode to preview changes
5. Provides detailed logging
6. Uses Click framework for CLI
7. Handles errors gracefully
"""

info("Example: Creating a CLI Tool")
code_block(f'requirements = """{cli_tool_requirements}"""\n\n# Use full workflow for CLI tool\nsession_id = submit_workflow(requirements, "full")')

### Example 3: System Architecture Design

In [ ]:
# Architecture Design Example
architecture_requirements = """
Design the architecture for a real-time collaborative document editing system similar to Google Docs:
1. Support multiple users editing simultaneously
2. Real-time synchronization
3. Conflict resolution
4. Version history
5. Scalable to millions of users
6. Include database schema
7. Define API contracts
8. Security considerations
"""

info("Example: System Architecture Design")
code_block(f'requirements = """{architecture_requirements}"""\n\n# Use individual workflow for just design\nsession_id = submit_workflow(requirements, "design")')

## 🐳 Docker Usage

The system can be run entirely in Docker for consistent environments.

In [ ]:
# Docker commands reference
docker_commands = {
    "Start all services": "docker-compose up",
    "Start in background": "docker-compose up -d",
    "View logs": "docker-compose logs -f",
    "Run example": "docker-compose exec orchestrator python hello_agents.py",
    "Run tests": "docker-compose exec orchestrator ./test_runner.py",
    "Stop services": "docker-compose down",
    "Clean everything": "docker-compose down -v"
}

info("Docker Commands Reference:")
for desc, cmd in docker_commands.items():
    display(HTML(f"<b>{desc}:</b> <code>{cmd}</code>"))

## 📊 Monitoring & Visualization

The system provides comprehensive monitoring and visualization capabilities.

In [ ]:
# Simulate workflow metrics visualization
import matplotlib.pyplot as plt
import numpy as np

# Sample metrics data
agents = ['Planner', 'Designer', 'Coder', 'Test Writer', 'Reviewer', 'Executor']
execution_times = [2.5, 3.8, 12.4, 8.2, 1.5, 5.6]
output_sizes = [1200, 2800, 15600, 8900, 450, 1200]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Execution time chart
ax1.bar(agents, execution_times, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FECA57', '#54A0FF'])
ax1.set_ylabel('Execution Time (seconds)')
ax1.set_title('Agent Execution Times')
ax1.tick_params(axis='x', rotation=45)

# Output size chart
ax2.pie(output_sizes, labels=agents, autopct='%1.1f%%', startangle=90)
ax2.set_title('Agent Output Distribution')

plt.tight_layout()
plt.show()

info("This visualization shows typical agent performance metrics")

## 🧪 Testing Capabilities

The system includes comprehensive testing features to ensure code quality.

In [ ]:
# Test categories and commands
test_categories = [
    {"Category": "Unit Tests", "Command": "./test_runner.py unit", "Description": "Fast, isolated component tests"},
    {"Category": "Integration Tests", "Command": "./test_runner.py integration", "Description": "Component interaction tests"},
    {"Category": "Workflow Tests", "Command": "./test_runner.py workflow", "Description": "End-to-end workflow validation"},
    {"Category": "Agent Tests", "Command": "./test_runner.py agent", "Description": "Individual agent functionality"},
    {"Category": "All Tests", "Command": "./test_runner.py", "Description": "Run complete test suite"}
]

df_tests = pd.DataFrame(test_categories)
display(HTML("<h4>Available Test Suites:</h4>"))
display(HTML(df_tests.to_html(index=False, classes="table table-striped")))

## 💡 Tips & Best Practices

Here are some tips to get the most out of the Multi-Agent Coding System:

In [ ]:
# Best practices
tips = [
    {
        "title": "📝 Clear Requirements",
        "tip": "Be specific about what you want. Include technical details, frameworks, and constraints.",
        "example": "Instead of 'Create a calculator', say 'Create a Calculator class in Python with add, subtract, multiply, divide methods, error handling for division by zero, and pytest tests'"
    },
    {
        "title": "🔄 Choose the Right Workflow",
        "tip": "Use TDD for critical code, Full for general apps, Individual steps for specific needs.",
        "example": "Use TDD workflow for APIs and libraries where test coverage is crucial"
    },
    {
        "title": "🎯 Iterative Development",
        "tip": "Start simple and iterate. Use the incremental workflow for adding features.",
        "example": "First create basic functionality, then add features like authentication, caching, etc."
    },
    {
        "title": "📊 Monitor Progress",
        "tip": "Use the real-time output feature to track agent activities and catch issues early.",
        "example": "Watch for reviewer rejections and understand why to improve your requirements"
    },
    {
        "title": "🧪 Leverage Testing",
        "tip": "The system writes comprehensive tests. Run them to ensure code quality.",
        "example": "Always check the generated tests and run them before deploying"
    }
]

for tip in tips:
    display(HTML(f"""
    <div style="border: 1px solid #ddd; padding: 10px; margin: 10px 0; border-radius: 5px;">
        <h4>{tip['title']}</h4>
        <p><strong>Tip:</strong> {tip['tip']}</p>
        <p><em>Example:</em> {tip['example']}</p>
    </div>
    """))

## 🎯 Interactive Workflow Builder

Let's create an interactive widget to build and submit workflows easily!

In [ ]:
# Create interactive workflow builder
def create_workflow_builder():
    # Widgets
    requirements_text = widgets.Textarea(
        value='Create a Python function to calculate the factorial of a number',
        placeholder='Enter your requirements here...',
        description='Requirements:',
        layout=widgets.Layout(width='100%', height='100px')
    )
    
    workflow_dropdown = widgets.Dropdown(
        options=['tdd', 'full', 'planning', 'design', 'implementation'],
        value='tdd',
        description='Workflow:',
    )
    
    submit_button = widgets.Button(
        description='Submit Workflow',
        button_style='success',
        icon='rocket'
    )
    
    output_area = widgets.Output()
    
    def on_submit(b):
        with output_area:
            clear_output()
            requirements = requirements_text.value
            workflow = workflow_dropdown.value
            
            info(f"Submitting {workflow} workflow...")
            info(f"Requirements: {requirements[:100]}...")
            
            # Here you would actually submit the workflow
            # session_id = submit_workflow(requirements, workflow)
            
            success("Workflow submitted! (In a real scenario, this would start the agents)")
    
    submit_button.on_click(on_submit)
    
    # Layout
    form = widgets.VBox([
        widgets.HTML("<h3>🚀 Workflow Builder</h3>"),
        requirements_text,
        workflow_dropdown,
        submit_button,
        output_area
    ])
    
    return form

# Display the workflow builder
workflow_builder = create_workflow_builder()
display(workflow_builder)

## 📈 Performance Metrics Dashboard

Let's create a dashboard to visualize system performance metrics.

In [ ]:
# Create performance metrics dashboard
def create_metrics_dashboard():
    # Sample data for demonstration
    timestamps = pd.date_range(start='2024-01-01 10:00', periods=20, freq='5min')
    
    metrics_data = {
        'timestamp': timestamps,
        'workflows_completed': np.cumsum(np.random.randint(1, 5, 20)),
        'avg_completion_time': np.random.uniform(30, 120, 20),
        'success_rate': np.random.uniform(85, 100, 20),
        'agents_active': np.random.randint(1, 7, 20)
    }
    
    df_metrics = pd.DataFrame(metrics_data)
    
    # Create subplots
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('Multi-Agent System Performance Dashboard', fontsize=16)
    
    # Workflows completed over time
    axes[0, 0].plot(df_metrics['timestamp'], df_metrics['workflows_completed'], 'b-', linewidth=2)
    axes[0, 0].set_title('Cumulative Workflows Completed')
    axes[0, 0].set_xlabel('Time')
    axes[0, 0].set_ylabel('Count')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Average completion time
    axes[0, 1].plot(df_metrics['timestamp'], df_metrics['avg_completion_time'], 'r-', linewidth=2)
    axes[0, 1].set_title('Average Completion Time')
    axes[0, 1].set_xlabel('Time')
    axes[0, 1].set_ylabel('Seconds')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Success rate
    axes[1, 0].plot(df_metrics['timestamp'], df_metrics['success_rate'], 'g-', linewidth=2)
    axes[1, 0].set_title('Success Rate')
    axes[1, 0].set_xlabel('Time')
    axes[1, 0].set_ylabel('Percentage')
    axes[1, 0].set_ylim(80, 105)
    axes[1, 0].grid(True, alpha=0.3)
    
    # Active agents
    axes[1, 1].bar(df_metrics['timestamp'], df_metrics['agents_active'], 
                   color='purple', alpha=0.7, width=0.002)
    axes[1, 1].set_title('Active Agents')
    axes[1, 1].set_xlabel('Time')
    axes[1, 1].set_ylabel('Count')
    axes[1, 1].set_ylim(0, 8)
    axes[1, 1].grid(True, alpha=0.3)
    
    # Adjust layout
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    display(HTML("<h4>📊 Summary Statistics</h4>"))
    summary_stats = {
        "Total Workflows": df_metrics['workflows_completed'].iloc[-1],
        "Average Success Rate": f"{df_metrics['success_rate'].mean():.1f}%",
        "Average Completion Time": f"{df_metrics['avg_completion_time'].mean():.1f}s",
        "Peak Active Agents": df_metrics['agents_active'].max()
    }
    
    for key, value in summary_stats.items():
        display(HTML(f"<strong>{key}:</strong> {value}"))

# Display the dashboard
create_metrics_dashboard()

## 🎓 Conclusion

Congratulations! You've explored the complete Multi-Agent Coding System. Here's what we covered:

1. **✅ System Architecture** - Understanding how 6 AI agents collaborate
2. **✅ Basic Usage** - Running simple examples with different workflows
3. **✅ REST API** - Integrating the system into your applications
4. **✅ Real-World Examples** - Building APIs, CLI tools, and architectures
5. **✅ Docker Deployment** - Running everything in containers
6. **✅ Testing & Monitoring** - Ensuring quality and tracking progress
7. **✅ Best Practices** - Tips for effective usage

### 🚀 Next Steps

1. **Try it yourself** - Use the interactive workflow builder above
2. **Explore workflows** - Experiment with different workflow types
3. **Build something** - Create your own applications
4. **Contribute** - Help improve the system

### 📚 Additional Resources

- **Documentation**: See `CLAUDE.md` for detailed system documentation
- **Examples**: Check `EXAMPLES.md` for more code examples
- **Testing**: Read `TEST_GUIDE.md` for testing guidelines
- **API Docs**: Visit `http://localhost:8000/docs` for interactive API documentation

### 🙏 Thank You!

Thank you for exploring the Multi-Agent Coding System. Happy coding! 🎉

In [ ]:
# Final message
display(HTML("""
<div style="text-align: center; padding: 20px; background-color: #f0f0f0; border-radius: 10px; margin-top: 20px;">
    <h2 style="color: #333;">🚀 Ready to Build Something Amazing?</h2>
    <p style="font-size: 18px; color: #666;">The Multi-Agent Coding System is at your service!</p>
    <p style="font-size: 16px; color: #888;">Transform your ideas into code with the power of AI collaboration.</p>
</div>
"""))